# Script to get true ranks for model evaluation

1. Modify the user query
2. Click the play button (can take up to 10 seconds to complete) and scroll down to interface
3. Choose your top 3 relevant Dandiset options for the given query, ranking them based on Dandiset ID
4. Submit form and wait for submission completion message

In [78]:
# ----------------------------------------------------- #

user_query = "Are there any dandisets that include EEG data?" # Edit query string 

# ----------------------------------------------------- #

import sys
sys.path.append("..")

import json
import config
from dandi.dandiapi import DandiAPIClient, RemoteDandiset
from rest.clients.qdrant import QdrantClient
# from rest.clients.openai import OpenaiClient
from IPython.display import display, clear_output
import ipywidgets as widgets

# number of similar results to fetch (adjust based on model)
TOP_K = 6

# semantic "simple" search vs keyword-based "keyword" search (adjust based on model)
METHOD = "simple"


# Populate Qdrant
def populate_qdrant():
    with open("../data/qdrant_points.json", "r") as file:
        emb = json.load(file)
    qdrant_client = QdrantClient(host="https://906c3b3f-d3ff-4497-905f-2d7089487cf9.us-east4-0.gcp.cloud.qdrant.io", api_key=config.QDRANT_API_KEY)
    qdrant_client.create_collection(collection_name="dandi_collection")
    qdrant_client.add_points_to_collection(collection_name="dandi_collection", embeddings_objects=emb)
    return qdrant_client


# Get semantic search results
def get_similar_results(query: str, qdrant_client: QdrantClient):
    similar_results = qdrant_client.query_from_user_input(text=query, collection_name="dandi_collection", top_k=TOP_K)
    return similar_results

# def get_keyword_results(query: str, openai_client: OpenaiClient, qdrant_client: QdrantClient):
#     keywords = openai_client.keywords_extraction(user_input=query)
#     similar_results = get_similar_results(query="".join(keywords), qdrant_client=qdrant_client)
#     return similar_results


qdrant_client = populate_qdrant()
if METHOD == "simple":
    similar_results = get_similar_results(user_query, qdrant_client)
# else:
#     openai_client = OpenaiClient()
#     similar_results = get_keyword_results(user_query, openai_client, qdrant_client)
similar_results = [id for id, _ in similar_results]
print(f"Similar dandiset results via {METHOD}-based search collected")

# Display selected query
query_title = widgets.HTML(value=f"<b>Query:</b> {user_query}")
display(query_title)


# Display information about each relevant dandiset
dandisets_title = widgets.HTML(value="<b>Similar Dandiset Results (click DANDI:ID to learn more)</b>")
display(dandisets_title)
for dandiset_result in similar_results:
    dandiset_path = dandiset_result.split(":")[-1]
    dandiset_archive_link = f"https://dandiarchive.org/dandiset/{dandiset_path}"
    dandi_client = DandiAPIClient()
    dandiset_id, version_id = dandiset_path.split("/")
    dandiset = RemoteDandiset(dandi_client, dandiset_id, version_id)
    dandiset_title = dandiset.get_metadata().name
    dandiset_text = widgets.HTML(value=f"""<li><a href='{dandiset_archive_link}' target='_blank'>{dandiset_result}</a> (<b>Title:</b> {dandiset_title})</li>""")
    display(dandiset_text)


# Create form submission interface for user feedback
form_title = widgets.HTML(value="<b>Rank 3 Dandisets based on DANDI:ID</b>")
initial_value = "Choose a dandiset"
rank_dropdown = [widgets.Dropdown(options=[initial_value] + similar_results, description=f"Rank #{i+1}:", value=initial_value) for i in range(3)]
submit_button = widgets.Button(description="Submit Ranks")
output = widgets.Output()


def on_button_click(b):
    ranks = [rank.value for rank in rank_dropdown]
    if initial_value in ranks:
        print("Ensure each rank has a designated Dandiset!")
    elif len(set(ranks)) != len(ranks):
        print("Please avoid selecting the same Dandiset for multiple rankings!")
    else:            
        # Store prompt, corresponding (rank: int, dandiset_id: str) pair, top_k, method values

        # End script
        clear_output()
        print("Submission complete. Thank you!")


submit_button.on_click(on_button_click)

# Show display
display(form_title)
display(rank_dropdown[0], rank_dropdown[1], rank_dropdown[2])
display(submit_button)

Submission complete. Thank you!
